In [7]:
import cv2
print cv2.imread('79432.png')

[[[73 73 73]
  [76 76 76]
  [72 72 72]
  ..., 
  [62 62 62]
  [66 66 66]
  [67 67 67]]

 [[89 89 89]
  [70 70 70]
  [69 69 69]
  ..., 
  [67 67 67]
  [66 66 66]
  [70 70 70]]

 [[72 72 72]
  [70 70 70]
  [72 72 72]
  ..., 
  [67 67 67]
  [59 59 59]
  [73 73 73]]

 ..., 
 [[76 76 76]
  [71 71 71]
  [74 74 74]
  ..., 
  [64 64 64]
  [85 85 85]
  [70 70 70]]

 [[72 72 72]
  [74 74 74]
  [75 75 75]
  ..., 
  [66 66 66]
  [78 78 78]
  [65 65 65]]

 [[73 73 73]
  [71 71 71]
  [74 74 74]
  ..., 
  [64 64 64]
  [70 70 70]
  [66 66 66]]]


In [6]:
import numpy as np
a = np.ones((2,3))
print a>1

[[False False False]
 [False False False]]


In [ ]:
#grab 100 frames with aperture closed to obtain offset

if dev.IsFrontPanelEnabled():
    row = 160
    N_adc = 4
    N_adcCh = 3
    N_mux = 46
    col = N_adc*N_adcCh*N_mux
    datain128 = bytearray(262144)
    datain1 = bytearray(88320)
    im = np.zeros((row ,col), np.uint8)
    im1 = np.zeros((79 ,60), np.uint8)
    im2 = np.zeros((79 ,60), np.uint8)
    
    exp =3
    exposure = 10**exp
    
    while exp<4.9:
        offset1 = np.zeros((79,60))
        elsecounter = 0   
        rep = 0
        dev.SetWireInValue(0x11,int(exposure))
        dev.SetWireInValue(0x12,1)
        dev.SetWireInValue(0x13,0)
        dev.SetWireInValue(0x14,0)
        dev.SetWireInValue(0x15, 0xfff003ff)
        dev.UpdateWireIns()
        # assert reset signal to initialize the FIFO
        dev.SetWireInValue(0x10, 0xff, 0x01)
        dev.UpdateWireIns()
        # deactivate reset signal and activate counter.
        dev.SetWireInValue(0x10, 0x00, 0x01)
        dev.UpdateWireIns()
        time.sleep(0.01)
        while rep<120:
            dev.UpdateTriggerOuts()
            # If the FIFO is full, read everything and display one frame only
            if dev.IsTriggered(0x6A, 0x01) == True:
                dev.ReadFromPipeOut(0xA0, datain128)
                for i in range(row):
                    for j in range(N_adc):
                        for k in range(N_adcCh):
                            for l in range(N_mux):
                                im[row-1-i][col-1-(j*N_adcCh*N_mux+(2-k)*N_mux+45-l)] = datain128[i*col+l*N_adc*N_adcCh+k*N_adc+j]
                #im = im/255
                im1[:,:] = im[1:80,142:262:2]
                if np.any(im1):
                    rep+=1
                    if rep>20: #to account for noise at the beginning
                        offset1 +=im1

            # If one frame is ready in FIFO
            elif dev.IsTriggered(0x6A, 0x02) == True:
                dev.ReadFromPipeOut(0xA0, datain1)
                for i in range(row):
                    for j in range(N_adc):
                        for k in range(N_adcCh):
                            for l in range(N_mux):
                                im[row-1-i][col-1-(j*N_adcCh*N_mux+(2-k)*N_mux+45-l)] = datain1[i*col+l*N_adc*N_adcCh+k*N_adc+j]
                #im = im/255
                im1[:,:] = im[1:80,142:262:2]
                rep+=1
                if np.any(im1):
                    if rep>20: #to account for noise at the beginning
                        offset1 +=im1
            else:
                elsecounter += 1
                print elsecounter
                if elsecounter>10:
                    elsecounter = 0
                    # assert reset signal to initialize the FIFO
                    dev.SetWireInValue(0x10, 0xff, 0x01)
                    dev.UpdateWireIns()
                    # deactivate reset signal and activate counter.
                    dev.SetWireInValue(0x10, 0x00, 0x01)
                    dev.UpdateWireIns()

            if (rep)%10 ==0:
                print rep 
        offset1= offset1/100
        cv2.imwrite(str(int(exposure))+'.png',offset1) #save offset images
        exp +=0.1
        exposure = 10**exp
else:
    print("FrontPanel host interface not detected.")